In [1]:
# %pip install langchain
# %pip install langchain-openai
# %pip install langchain-community
# %pip install langchainhub
# %pip install pypdf
# %pip install faiss-cpu

For this notebook besides the OPENAI API KEY, you will need the LANGCHAIN API KEY (if you want to use langsmith platform for evaluation) and the TAVILY API KEY. 

1. [See here for the quick setup for LANGSMITH](https://docs.smith.langchain.com/).
2. [See here for the quick setup for TAVILY API KEY](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/).

In [2]:
import os
import getpass

# Set OPENAI API Key

import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# os.environ["TAVILY_API_KEY"] = getpass.getpass() # you can obtain your tavily api key here: https://tavily.com/
os.environ["LANGCHAIN_TRACING_V2"] = "true"
_set_env("LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_PROJECT="default"

Create your api key in the settings page of the [langsmith platform here](https://smith.langchain.com/settings).

> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

# The Modern Tool Calling Way of LangChain

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [multiply, exponentiate, add]


llm = ChatOpenAI(model="gpt-4o", temperature=0)


agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2.743}`


82.65606421491815
Invoking: `add` with `{'x': 3, 'y': 5}`


8.0The result of \(3 + 5^{2.743}\) is approximately \(82.656\).

For the subtraction \(17.24 - 918.1241\), the result is \(-900.8841\).

> Finished chain.


{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': 'The result of \\(3 + 5^{2.743}\\) is approximately \\(82.656\\).\n\nFor the subtraction \\(17.24 - 918.1241\\), the result is \\(-900.8841\\).'}

source: https://arc.net/l/quote/cdomnqvm

So when should you use with_structured_output versus binding tools and reading tool calls directly?

with_structured_output always returns a structured output in the schema that you specified. This is useful when you want to force the LLM to output information that matches a specific schema. This is useful for information extraction tasks.

bind_tools is more general and can select a specific tool - or no tool, or multiple tools! This is useful when you want to allow the LLM to have more flexibility in how it should respond - for example, in agent applications where you need to choose which tools to invoke but also respond to the user.

**Tool Calling Also with Pydantic**

In [4]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )




llm_with_structured_output = llm.with_structured_output(Joke)

In [5]:
llm_with_structured_output.invoke("Joke about how cats are evil")

Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!', rating=7)

# Tool Calling

In [6]:
from langchain_openai import ChatOpenAI
def send_error_message(message: str) -> str:
    """Send an error message to the user."""
    return f"Error: {message}"

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_tools = llm.bind_tools([send_error_message])
llm_tools.invoke("If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever.")
# Pydantic


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7Miyd4bVqymRH9WZMIzTDLLn', 'function': {'arguments': '{"message":"Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don\'t cover \'universal domination\' in their warranty!"}', 'name': 'send_error_message'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 93, 'total_tokens': 154, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e33874f-0849-4e5d-87cc-3cbddcb68e53-0', tool_calls=[{'name': 'send_error_message', 'args': {'message': "Becoming the master of the universe 

In [7]:
class send_error_message(BaseModel):
    """Send an error message to the user."""
    message: str = Field(description="The error message to send to the user")


llm_tools = llm.bind_tools([send_error_message])

In [8]:
llm_tools.invoke("If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever.")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9cdLiW7EbFT7VymtlnPpJhim', 'function': {'arguments': '{"message":"Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don\'t cover \'universal domination\' in their warranty!"}', 'name': 'send_error_message'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 103, 'total_tokens': 164, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_818c284075', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-30fe0dd2-6c04-48c9-9c01-f2725bbc405f-0', tool_calls=[{'name': 'send_error_message', 'args': {'message': "Becoming the master of the universe

In [9]:
from langchain_core.output_parsers import PydanticToolsParser


query = "If the following request is impossible to achieve, send an error message to the user with a funny punchline:\
    My request is that I become the master of the universe with all the money and intelligence that surpasses any human ever."

chain = llm_tools | PydanticToolsParser(tools=[send_error_message])
chain.invoke(query)

[send_error_message(message="Becoming the master of the universe is a tall order! But hey, if you find a genie, just make sure to read the fine print—last time I checked, they don't cover 'universal domination' in their warranty!")]

In [10]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [11]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_UZ98HyLbgzZRH0RSnZ2Ohqd7', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_rkv4W0VlqSJxFtKv5SCleCXB', 'type': 'tool_call'}]


In [12]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UZ98HyLbgzZRH0RSnZ2Ohqd7', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_rkv4W0VlqSJxFtKv5SCleCXB', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 87, 'total_tokens': 137, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-066b1abc-56ad-4846-b507-061e7adfaec7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_UZ9

In [13]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 12\\) is 36, and the result of \\(11 + 49\\) is 60.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 153, 'total_tokens': 184, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae55e290-b722-4865-a57b-49739f46a336-0', usage_metadata={'input_tokens': 153, 'output_tokens': 31, 'total_tokens': 184, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

![](2023-10-27-19-28-10.png)

In [14]:
MODEL='gpt-4o-mini'
TEMP=0.0

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [16]:
search = TavilySearchResults()

In [17]:
search.invoke("what is the weather in Paris?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.8667, 'lon': 2.3333, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1733933640, 'localtime': '2024-12-11 17:14'}, 'current': {'last_updated_epoch': 1733932800, 'last_updated': '2024-12-11 17:00', 'temp_c': 4.3, 'temp_f': 39.7, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 53, 'wind_dir': 'NE', 'pressure_mb': 1028.0, 'pressure_in': 30.36, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 81, 'cloud': 100, 'feelslike_c': 1.4, 'feelslike_f': 34.6, 'windchill_c': 3.3, 'windchill_f': 37.9, 'heatindex_c': 5.8, 'heatindex_f': 42.5, 'dewpoint_c': 1.3, 'dewpoint_f': 34.3, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 10.2, 'gust_kph': 16.3}}"},
 {'url': 'https://www.weathertab.com/en/c/e/12/republic-of-france/ile-de-france/p

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow t

In [19]:
docs[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to

In [20]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

documents

[Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild a

In [21]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [22]:
retriever = vector.as_retriever()

In [23]:
retriever.invoke("What is langchain?")[0]

Document(metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:')

In [24]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [25]:
tools = [search, retriever_tool]

In [26]:
from langchain_openai import ChatOpenAI

MODEL="gpt-4o-mini"
llm = ChatOpenAI(model=MODEL, temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d0751bea-d748-4622-8ca1-26f226b71f91-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={})

In [29]:
prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [30]:
prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})

In [31]:
prompt.messages[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [32]:
tools

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 Tool(name='langchain_search', description='Search for information about LangChain. For any questions about LangChain, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x114522e80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x123dcda90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1145234c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x123dcda90>, search_kwargs={}), document_prompt=PromptTemplate(input_va

In [33]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [34]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [35]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain? List 5 in bullet points."})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations.
Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.
Productionization: Use LangSmith to inspect, monitor and evaluate your chains, so that you can continuously optimize and deploy with confidence.
Deployment: Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Platform.



Concretely, the framework consists of the following open-source libraries:

Introduction | 🦜️🔗 LangChain

langchain-core: Base abstractions and LangChain Expression Language.
Integration packages (e.g. langchain-openai, langchain-

{'input': 'What are some useful applications you can build with LangChain? List 5 in bullet points.',
 'output': 'Here are five useful applications you can build with LangChain:\n\n- **Conversational Agents**: Create chatbots or virtual assistants that can engage in natural language conversations, providing customer support or information retrieval.\n\n- **Document Analysis Tools**: Develop applications that can analyze and summarize large documents, extracting key information and insights for users.\n\n- **Stateful Multi-Actor Applications**: Build complex applications that involve multiple agents interacting with each other, using LangGraph to model their interactions.\n\n- **API Development**: Turn your LangChain applications into production-ready APIs, allowing other applications to leverage your LLM capabilities.\n\n- **Monitoring and Evaluation Tools**: Use LangSmith to inspect, monitor, and evaluate your LLM applications, ensuring they perform optimally and meet user needs.'}

Now we can inspect what is going on in the [langsmith platform!](https://smith.langchain.com/o/b68c89e6-dae0-54ab-a84b-3201d39a3c50/settings).